In [ ]:
from evaluateResults import makeTestResultTable
from pathlib import PosixPath as Path

root = Path("..")

tableDir = root / "evaluation" / "tables"

for optimized in (True, False):
    optStr = "optimized" if optimized else "unoptimized"
    for testNum in (1, 2):
        fileName = f"{optStr}-test{testNum}"
        table, df = makeTestResultTable(testNum=testNum, optimized=optimized, root=root)
        with Path.open(tableDir / "tex" / f"{fileName}.tex", "w") as f:
            f.write(table.render())
        df.to_csv(tableDir / "csv" / f"{fileName}.csv")